# 택시 데이터를 활용한 수요 예측 모델
## 현재목표
택시데이터를 분류하는 모델 생각하기

### 기준
    -공간 기준: x,y_좌표 간격 = 0.001
    -시간 기준: 5분 간격
### todo
지금 할일

In [2]:
#라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from sklearn import datasets

In [5]:
#예시 데이터 1개
df = pd.read_csv("/Users/joyumin/PycharmProjects/datas/taxi.csv", encoding="euc-kr")
print("원본", df.shape)
df = df.dropna(axis=0)
print("NAN 제거", df.shape)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '_').str.replace(')', '')

원본 (59326905, 3)
NAN 제거 (59326905, 3)


In [ ]:
df['승차시각'] = pd.to_datetime(df['승차시각'])
# df = df.query("승차위치_시도 == '서울특별시'")

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59326905 entries, 0 to 59326904
Data columns (total 5 columns):
승차시각          object
승차위치_x좌표      float64
승차위치_y좌표      float64
time_label    int64
loc_idx       int64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.7+ GB


In [7]:
df['count'].value_counts()

1     28085669
2     13073624
3      6594312
4      3676924
5      2233220
6      1446792
7       990934
8       710536
9       524115
10      394210
11      304865
12      243972
13      194051
14      155176
15      128175
16      107136
17       87482
18       71190
19       60458
20       50260
21       42378
22       32780
23       27646
24       20664
25       16325
26       12896
27        9315
28        7420
29        5858
30        3990
31        2790
33        2706
32        1984
34        1428
35         910
37         851
36         684
40         640
38         494
39         468
42         462
41         287
44         264
43         172
48         144
53         106
50          50
47          47
45          45
Name: count, dtype: int64

# 시간

#### 5분단위로 타임 카테고리를 만듬
#### 승차시각을 초로 환산해서 300(5분)초로 나눈값을 저장함

In [11]:
df2 = df.filter(["승차시각", "승차위치_x좌표", "승차위치_y좌표", "승차위치_시도"])

In [12]:
# df2['승차_일'] = [d.date() for d in df2['승차시각']]
# df2['승차_시간'] = [d.time() for d in df2['승차시각']]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6482458 entries, 0 to 7099266
Data columns (total 4 columns):
승차시각        datetime64[ns]
승차위치_x좌표    float64
승차위치_y좌표    float64
승차위치_시도     object
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 247.3+ MB


In [13]:
#5분 단위로 쪼갬 카운트
hour_data = df2.groupby(pd.Grouper(key='승차시각',freq='5min'))['승차시각'].count()
hour_data

승차시각
2018-01-29 00:00:00    3767
2018-01-29 00:05:00    3806
2018-01-29 00:10:00    3611
2018-01-29 00:15:00    3176
2018-01-29 00:20:00    3150
2018-01-29 00:25:00    2923
2018-01-29 00:30:00    2886
2018-01-29 00:35:00    2732
2018-01-29 00:40:00    2562
2018-01-29 00:45:00    2576
2018-01-29 00:50:00    2486
2018-01-29 00:55:00    2368
2018-01-29 01:00:00    2380
2018-01-29 01:05:00    2409
2018-01-29 01:10:00    2231
2018-01-29 01:15:00    2228
2018-01-29 01:20:00    2025
2018-01-29 01:25:00    2003
2018-01-29 01:30:00    1933
2018-01-29 01:35:00    1885
2018-01-29 01:40:00    1860
2018-01-29 01:45:00    1836
2018-01-29 01:50:00    1730
2018-01-29 01:55:00    1812
2018-01-29 02:00:00    1862
2018-01-29 02:05:00    1814
2018-01-29 02:10:00    1771
2018-01-29 02:15:00    1581
2018-01-29 02:20:00    1501
2018-01-29 02:25:00    1500
                       ... 
2018-02-04 21:30:00    3285
2018-02-04 21:35:00    3474
2018-02-04 21:40:00    3451
2018-02-04 21:45:00    3335
2018-02-04 21:5

In [5]:
#승차시각을 초로 환산해서 300초(5분)씩 나눔
s = (df2.승차시각 - df2.승차시각.dt.floor('D')).dt.total_seconds() / 300
# s = pd.to_numeric(s)
s = s.astype(int)
df2['time_label'] = s
df2.head()

,승차시각,승차위치_x좌표,승차위치_y좌표,승차위치_시도,time_label
0,2018-03-05 00:00:00,127.104445,37.484585,서울특별시,0
1,2018-03-05 00:00:00,127.115560,37.496255,서울특별시,0
2,2018-03-05 00:00:00,127.084957,37.537818,서울특별시,0
3,2018-03-05 00:00:01,127.057588,37.543923,서울특별시,0
4,2018-03-05 00:00:01,127.087842,37.557823,서울특별시,0


# 공간
##### 어림 잡은 값으로 나눠도 상관없을 거 같음
### 0.0001은 대략 10미터 <br> 따라서 기준은 0.0025로 잡기로 함 (250m) <br> y = 0.0012 100미터 <br> y = 기준은 .003
#### 서울시 기준
----------

동 - 강동구 - 강일동 : 37.545826 ,*127.183551* <br>
서 - 강서구 - 공항동 : 37.555339, *126.764278* <br>
남 - 서초구 - 내곡동 : *37.429010*, 127.065674 <br>
북 - 도봉구 - 도봉1동 : *37.70107* 9, 127.019567 

##### 이렇게 되면 낭비되는 공간이 많이 생기는데 그 부분에 대해서 생각해 봐야댐

In [6]:
temp_df = df.filter(["승차시각", "승차위치_x좌표", "승차위치_y좌표", "승차위치_시도"])

## 지도를 나누고 인덱스 구하기
##### 기준이  0.001이기 떄문에 소수점 3자리까지 내림하고 계산
(x 최대 - x 최소) / 0.0035 = 119 (x의 인덱스는 168까지 있음) <br>
(y 최대 - y 최소) / 0.00227= 272 (Y의 인덱스는 118까지 있음)

### 내 위치의 인덱스 구하기
(x 내위치 - x 최소) / 0.001 = 내 위치 인덱스 (y도 같은 방법으로)

### 카테리고리 컬럼 정하기
----

In [7]:
#모든 구역에 인덱스를 정해주기
#시간이 오래 걸린다는 단점이 있음
#x + (n / y)
loc_df = temp_df
def getSpaceIndex(x, y):
    x_idx = int((num_floor(x, 3) - 126.764) / 0.001)
    y_idx = int((num_floor(y, 3) - 37.429) / 0.001)
    
    return x_idx + (419 * y_idx)

def num_floor(num, f=0):
    f = 10 ** f
    temp = math.floor(num * f);
    return temp / f

df2["loc_idx"] = loc_df.apply(lambda row: getSpaceIndex(row.승차위치_x좌표, row.승차위치_y좌표), axis=1)
df2.head()

,승차시각,승차위치_x좌표,승차위치_y좌표,승차위치_시도,time_label,loc_idx
0,2018-03-05 00:00:00,127.104445,37.484585,서울특별시,0,22966
1,2018-03-05 00:00:00,127.115560,37.496255,서울특별시,0,28423
2,2018-03-05 00:00:00,127.084957,37.537818,서울특별시,0,45153
3,2018-03-05 00:00:01,127.057588,37.543923,서울특별시,0,47640
4,2018-03-05 00:00:01,127.087842,37.557823,서울특별시,0,53955


### 모든 구역에 키값 정해주기
###### 이 방법은 시간이 오래걸림 그리고 많이 부족한 느낌이 듬 뭔가 새로운 방법이 필요함
x_idx + (140 * y_idx) = 구역의 인덱스 <br>
x + (n * y)
##### 해독하기
원래 값 : a <br>
a % n = x <br>
(a - x) / n = y

------


In [ ]:
df2 = df.groupby(['time_label', 'loc_idx'])
df3 = pd.DataFrame(df2)
df3.head()

## 모델 학습하기
----------
### 시간

In [28]:
df['승차시각'] = [str(d.date()).replace("-", "") for d in df['승차시각']]
# df2['승차_시간'] = [str(d.time()).replace(":", "") for d in df2['승차시각']]

In [29]:
from sklearn.model_selection import train_test_split
x = df[['승차시각', '승차위치_x좌표', '승차위치_y좌표', 'loc_idx']] #예측의 기반이 될 컬럼
y = df['time_label'] #예측결과를 받을 컬럼
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
#feature = 속성

forest = RandomForestClassifier(n_estimators=100)
forest.fit(x_train, y_train)

y_pred = forest.predict(x_test) #예측한 결과 
# y_pred

print('정확도 : ', metrics.accuracy_score(y_test, y_pred))